In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_horizVert_wavelet_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,161840.78,-2.13,-1.50,-1.07,-0.18,-0.18,0.06,-0.07,-0.01
3,-2.13,35149.91,-0.57,0.11,0.05,-0.01,-0.01,0.00,0.01
4,-1.50,-0.57,8890.51,0.03,0.07,-0.01,-0.01,-0.00,-0.00
5,-1.07,0.11,0.03,1591.83,-0.01,-0.00,0.00,-0.01,-0.00
6,-0.18,0.05,0.07,-0.01,337.33,0.01,0.00,0.00,0.00
7,-0.18,-0.01,-0.01,-0.00,0.01,89.28,0.00,0.00,0.00
8,0.06,-0.01,-0.01,0.00,0.00,0.00,29.97,0.00,-0.00
9,-0.07,0.00,-0.00,-0.01,0.00,0.00,0.00,8.94,-0.00
10,-0.01,0.01,-0.00,-0.00,0.00,0.00,-0.00,-0.00,1.40


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00003,-0.00004,-0.00007,-0.00002,-0.00005,0.00003,-0.00005,-0.00001
3,-0.00003,1.00000,-0.00003,0.00001,0.00001,-0.00001,-0.00001,0.00001,0.00005
4,-0.00004,-0.00003,1.00000,0.00001,0.00004,-0.00001,-0.00003,-0.00001,-0.00001
5,-0.00007,0.00001,0.00001,1.00000,-0.00001,-0.00000,0.00001,-0.00006,-0.00005
6,-0.00002,0.00001,0.00004,-0.00001,1.00000,0.00004,0.00002,0.00002,0.00001
7,-0.00005,-0.00001,-0.00001,-0.00000,0.00004,1.00000,0.00001,0.00006,0.00000
8,0.00003,-0.00001,-0.00003,0.00001,0.00002,0.00001,1.00000,0.00000,-0.00000
9,-0.00005,0.00001,-0.00001,-0.00006,0.00002,0.00006,0.00000,1.00000,-0.00004
10,-0.00001,0.00005,-0.00001,-0.00005,0.00001,0.00000,-0.00000,-0.00004,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

4.089209555235006

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.91268636e+09 1.35226585e+08 8.46456426e+06 2.64231266e+05
 1.14168173e+04 7.55024600e+02 7.56784469e+01 4.99454307e+00
 2.20696555e-32]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999571,-0.028460,-0.006845,-0.001227,-0.000260,-0.000070,-0.000023,-0.000007,-0.000001
1,0.028166,0.998830,-0.038747,-0.006439,-0.001361,-0.000361,-0.000121,-0.000036,-0.000005
2,0.007891,0.038312,0.998730,-0.031061,-0.006339,-0.001678,-0.000565,-0.000168,-0.000026
3,0.001635,0.007507,0.030449,0.998448,-0.044447,-0.011172,-0.003736,-0.001119,-0.000176
4,0.000411,0.001885,0.007459,0.043222,0.996246,-0.070991,-0.021972,-0.006495,-0.001018
5,0.000133,0.000609,0.002406,0.013468,0.066931,0.989315,-0.124642,-0.032054,-0.004968
6,0.000059,0.000272,0.001077,0.006009,0.028505,0.115964,0.979146,-0.162728,-0.022239
7,0.000027,0.000124,0.000490,0.002739,0.012905,0.048987,0.152008,0.973893,-0.160788
8,0.000009,0.000039,0.000156,0.000872,0.004102,0.015501,0.046187,0.154861,0.986725


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00042931, 0.00116975, 0.0012699 , 0.00155206, 0.00375366,
       0.01068458, 0.02085363, 0.02610736, 0.01327457])